In [3]:
import pandas as pd

In [5]:
import pickle

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
!pip install pyarrow

In [8]:
df_yellow_01_23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_yellow_02_23 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

## Q1 - How many columns are there?

In [7]:
df_yellow_01_23.shape

(3066766, 19)

In [8]:
df_yellow_02_23.shape

(2913955, 19)

## Duration

In [9]:
df_yellow_01_23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [10]:
df_yellow_01_23['duration'] = df_yellow_01_23.tpep_dropoff_datetime - df_yellow_01_23.tpep_pickup_datetime
df_yellow_01_23.duration = df_yellow_01_23.duration.apply(lambda td: td.total_seconds() / 60)
df_yellow_01_23[['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'duration']]

,tpep_dropoff_datetime,tpep_pickup_datetime,duration
0,2023-01-01 00:40:36,2023-01-01 00:32:10,8.433333
1,2023-01-01 01:01:27,2023-01-01 00:55:08,6.316667
2,2023-01-01 00:37:49,2023-01-01 00:25:04,12.750000
3,2023-01-01 00:13:25,2023-01-01 00:03:48,9.616667
4,2023-01-01 00:21:19,2023-01-01 00:10:29,10.833333
...,...,...,...
3066761,2023-02-01 00:12:33,2023-01-31 23:58:34,13.983333
3066762,2023-01-31 23:50:36,2023-01-31 23:31:09,19.450000
3066763,2023-01-31 23:25:36,2023-01-31 23:01:05,24.516667
3066764,2023-01-31 23:53:00,2023-01-31 23:40:00,13.000000


In [10]:
df_yellow_02_23['duration'] = df_yellow_02_23.tpep_dropoff_datetime - df_yellow_02_23.tpep_pickup_datetime
df_yellow_02_23.duration = df_yellow_02_23.duration.apply(lambda td: td.total_seconds() / 60)
df_yellow_02_23[['tpep_dropoff_datetime', 'tpep_pickup_datetime', 'duration']]

,tpep_dropoff_datetime,tpep_pickup_datetime,duration
0,2023-02-01 00:34:34,2023-02-01 00:32:53,1.683333
1,2023-02-01 00:35:30,2023-02-01 00:35:16,0.233333
2,2023-02-01 00:35:30,2023-02-01 00:35:16,0.233333
3,2023-02-01 01:01:38,2023-02-01 00:29:33,32.083333
4,2023-02-01 00:25:46,2023-02-01 00:12:28,13.300000
...,...,...,...
2913950,2023-03-01 00:05:00,2023-02-28 23:46:00,19.000000
2913951,2023-02-28 23:37:10,2023-02-28 23:26:02,11.133333
2913952,2023-02-28 23:38:00,2023-02-28 23:24:00,14.000000
2913953,2023-02-28 23:10:00,2023-02-28 23:03:00,7.000000


In [11]:
df_yellow_02_23 = df_yellow_02_23[(df_yellow_02_23.duration >= 1) & (df_yellow_02_23.duration <= 60)]

In [12]:
categorical = ['PULocationID', 'DOLocationID']
df_yellow_02_23[categorical] = df_yellow_02_23[categorical].astype(str)

/tmp/ipykernel_23016/3282935156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yellow_02_23[categorical] = df_yellow_02_23[categorical].astype(str)


In [11]:
# Q2 - What's the standard deviation of the trips duration in January?
df_yellow_01_23['duration'].std()

42.594351241920904

## Outliers

In [12]:
df_yellow_01_23_clean = df_yellow_01_23[(df_yellow_01_23.duration >= 1) & (df_yellow_01_23.duration <= 60)]

print(len(df_yellow_01_23_clean['duration']))

3009173


In [13]:
# Q3 - What fraction of the records left after you dropped the outliers?
print(len(df_yellow_01_23_clean['duration'])/len(df_yellow_01_23['duration'])*100)

98.1220282212598


## One Hot Encoding

In [14]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['duration']

df_yellow_01_23_clean[categorical] = df_yellow_01_23_clean[categorical].astype(str)

/tmp/ipykernel_21771/23254822.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yellow_01_23_clean[categorical] = df_yellow_01_23_clean[categorical].astype(str)


In [15]:
train_dicts = df_yellow_01_23_clean[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [16]:
# What's the dimensionality of this matrix (number of columns)?
X_train.shape

(3009173, 515)

## Train Model

In [17]:
target = 'duration'
y_train = df_yellow_01_23_clean[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [18]:
# What's the RMSE on train?
mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605

## Evaluate the model

In [9]:
def clean_dataframe(df):
    print('computing duration')
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    print('removing outliers')
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    #print('convert ID to string')
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df = df[['PULocationID', 'DOLocationID','duration']]
    
    return df

In [10]:
df_train = clean_dataframe(df_yellow_01_23)

computing duration
removing outliers


/tmp/ipykernel_32927/778110110.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [11]:
df_val = clean_dataframe(df_yellow_02_23)

computing duration
removing outliers


/tmp/ipykernel_32927/778110110.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [10]:
df_val.shape

(2855951, 3)

In [11]:
df_train.shape

(3009173, 3)

In [12]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

/tmp/ipykernel_32927/652578202.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']


In [13]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)


In [14]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.198572035205691